In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
df_train = pd.read_csv('/kaggle/input/fake-news/train.csv')
df_test=pd.read_csv('/kaggle/input/fake-news/test.csv')

In [4]:
x_train=df_train.drop(columns=['id','label'])
x_test=df_test.drop(columns=['id'])

In [5]:
y=df_train.label

In [6]:
x_train.info(),x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20242 non-null  object
 1   author  18843 non-null  object
 2   text    20761 non-null  object
dtypes: object(3)
memory usage: 487.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   5078 non-null   object
 1   author  4697 non-null   object
 2   text    5193 non-null   object
dtypes: object(3)
memory usage: 122.0+ KB


(None, None)

In [7]:
x_train=x_train.fillna('')
x_test=x_test.fillna('')

In [8]:
x_train.info(),x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   20800 non-null  object
 1   author  20800 non-null  object
 2   text    20800 non-null  object
dtypes: object(3)
memory usage: 487.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   5200 non-null   object
 1   author  5200 non-null   object
 2   text    5200 non-null   object
dtypes: object(3)
memory usage: 122.0+ KB


(None, None)

In [9]:
x_train['content'] = x_train['author']+' '+x_train['title']
x_test['content'] = x_test['author']+' '+x_test['title']

In [10]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [11]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space]
    tokens = [token for token in tokens if not token in nlp.Defaults.stop_words]
    preprocessed_text = " ".join(tokens)
    return preprocessed_text

In [12]:
x_train['content']=x_train['content'].apply(preprocess_text)

In [13]:
x_test['content']=x_test['content'].apply(preprocess_text)

In [14]:
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(x_train['content'])
X_test=vectorizer.transform(x_test['content'])

In [15]:
from sklearn.svm import SVC
model=SVC(kernel='linear',C=1.0)

In [16]:
model.fit(X_train,y)

SVC(kernel='linear')

In [17]:
model.score(X_train,y)

0.9979807692307693

In [32]:
y_pred=model.predict(X_test)

In [33]:
df_submit = pd.read_csv('/kaggle/input/fake-news/submit.csv')

In [35]:
df_submit['label'] = y_pred
df_submit['label'] = df_submit['label'].astype(int)

In [38]:
df_submit.to_csv('submission.csv', index=False)